# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [2]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from prettytable import PrettyTable

#### Creating list of filepaths to process original event csv data files

In [3]:
#get current path then add /event_data 
print(os.getcwd())

filepath = os.getcwd() + '/event_data'
for root, dirs, files in os.walk(filepath):
        file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [8]:
from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [9]:
# Create a Keyspace 
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS sparkify 
        WITH REPLICATION = 
        { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [10]:
# Set KEYSPACE
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [11]:
## Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

select_by_session_item = """
    SELECT artist, song, length
    FROM events_by_session_item
    WHERE sessionid = 338 AND iteminsession = 4;
"""

create_by_session_item = "CREATE TABLE IF NOT EXISTS events_by_session_item "
create_by_session_item = create_by_session_item + \
    "(sessionid int, iteminsession int, artist text, song text, length decimal, \
    PRIMARY KEY (sessionid, iteminsession))"
try:
    session.execute(create_by_session_item)
except Exception as e:
    print(e)

In [13]:
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
##Assign the INSERT statements into the `query` variable
        insert_by_session_item = """
            INSERT INTO events_by_session_item (sessionid, iteminsession, artist, song, length)
        """
        insert_by_session_item = insert_by_session_item + " VALUES (%s, %s, %s, %s, %s)"
        try:
            session.execute(
                insert_by_session_item,
                (int(line[8]), int(line[3]), line[0], line[9], float(line[5]))
            )
        except Exception as e:
            print(e)

#### Do a SELECT to verify that the data have been inserted into each table

In [14]:
## Add in the SELECT statement to verify the data was entered into the table
try:
    rows = session.execute(select_by_session_item)
except Exception as e:
    print(e)
t = PrettyTable(["Artist", "Song", "Length"])
for row in rows:
    t.add_row([row.artist, row.song, str(round(row.length, 2)) + " secs"])
print("Table 1. Song Info for Session ID: 338, Item: 4")
print(t)

Table 1. Song Info for Session ID: 338, Item: 4
+-----------+---------------------------------+-------------+
|   Artist  |               Song              |    Length   |
+-----------+---------------------------------+-------------+
| Faithless | Music Matters (Mark Knight Dub) | 495.31 secs |
+-----------+---------------------------------+-------------+


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [15]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

#create select statment
select_by_user_session_item = """
    SELECT artist, song, firstname, lastname
    FROM events_by_user_session_item
    WHERE userid = 10 AND sessionid = 182;
"""

create_by_user_session_item = "CREATE TABLE IF NOT EXISTS events_by_user_session_item "
create_by_user_session_item = create_by_user_session_item + \
    "(userid int, sessionid int, iteminsession int, artist text, song text, \
    firstname text, lastname text, PRIMARY KEY ((userid, sessionid), iteminsession))"
try:
    session.execute(create_by_user_session_item)
except Exception as e:
    print(e)
                    

In [16]:
# Insert data into tables
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        insert_by_user_session_item = """
            INSERT INTO events_by_user_session_item (
                userid, sessionid, iteminsession, artist, song, firstname, lastname
            )
        """
        insert_by_user_session_item = insert_by_user_session_item + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        try:
            session.execute(
                insert_by_user_session_item,
                (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4])
            )
        except Exception as e:
            print(e)

In [17]:
# Run and print output of select queries for table 2
try:
    rows = session.execute(select_by_user_session_item)
except Exception as e:
    print(e)
t = PrettyTable(["Artist", "Song", "First Name", "Last Name"])
for row in rows:
    t.add_row([row.artist, row.song, row.firstname, row.lastname])
print("Table 2. All Songs Played for User ID: 10, Session ID: 182")
print(t)

Table 2. All Songs Played for User ID: 10, Session ID: 182
+-------------------+------------------------------------------------------+------------+-----------+
|       Artist      |                         Song                         | First Name | Last Name |
+-------------------+------------------------------------------------------+------------+-----------+
|  Down To The Bone |                  Keep On Keepin' On                  |   Sylvie   |    Cruz   |
|    Three Drives   |                     Greece 2000                      |   Sylvie   |    Cruz   |
| Sebastien Tellier |                      Kilometer                       |   Sylvie   |    Cruz   |
|   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |   Sylvie   |    Cruz   |
+-------------------+------------------------------------------------------+------------+-----------+


In [14]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#create select stament
select_by_song_user = """
    SELECT firstname, lastname
    FROM events_by_song_user
    WHERE song = 'All Hands Against His Own';
"""
#create tab
create_by_song_user = "CREATE TABLE IF NOT EXISTS events_by_song_user "
create_by_song_user = create_by_song_user + \
    "(song text, userid int, firstname text, lastname text, sessionid int, iteminsession int, \
    PRIMARY KEY (song, userid))"
try:
    session.execute(create_by_song_user)
except Exception as e:
    print(e)               

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        insert_by_song_user = """
            INSERT INTO events_by_song_user (
                song, userid, firstname, lastname, sessionid, iteminsession
            )
        """
        insert_by_song_user = insert_by_song_user + " VALUES (%s, %s, %s, %s, %s, %s)"
        try:
            session.execute(
                insert_by_song_user,
                (line[9], int(line[10]), line[1], line[4], int(line[8]), int(line[3]))
            )
        except Exception as e:
            print(e)

In [16]:
# Run and print output of select queries
try:
    rows = session.execute(select_by_song_user)
except Exception as e:
    print(e)
t = PrettyTable(["First Name", "Last Name"])
for row in rows:
    t.add_row([row.firstname, row.lastname])
print("Table 3. All Users Who Have Listened to 'All Hands Against His Own'")
print(t)

Table 3. All Users Who Have Listened to 'All Hands Against His Own'
+------------+-----------+
| First Name | Last Name |
+------------+-----------+
| Jacqueline |   Lynch   |
|   Tegan    |   Levine  |
|    Sara    |  Johnson  |
+------------+-----------+


### Drop the tables before closing out the sessions

In [17]:
## TO-DO: Drop the table before closing out the sessions
try:
    rows = session.execute("DROP TABLE events_by_session_item")
except Exception as e:
    print(e)
try:
    rows = session.execute("DROP TABLE events_by_user_session_item")
except Exception as e:
    print(e)
try:
    rows = session.execute("DROP TABLE events_by_song_user")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()